Toy example: 

In [1]:
import pandas as pd

In [7]:
Y = pd.Series([1, 2, 2.5, 1.2, 3, 6, 3.5, 2.4, 1.2, 1.5, 1.6, 4])
X1 = pd.Series([-2, -2, -5, -3, -3, 7, 5, 4, -2, 1, 1, 2])
X2 = pd.Series([41, 26, 32, 27, 31, 17, 30, 25, 20, 5, 8.6, 40])
X3 = pd.Series([-1, 22, 22.5, 3.2, 4, 2, 5, 4, 2, 5, 6, 4])

In [8]:
Data = pd.DataFrame({'Y':Y , 'X1':X1, 'X2':X2, 'X3':X3})

In [32]:
Data

,Y,X1,X2,X3
0,1.0,-2,41.0,-1.0
1,2.0,-2,26.0,22.0
2,2.5,-5,32.0,22.5
3,1.2,-3,27.0,3.2
4,3.0,-3,31.0,4.0
5,6.0,7,17.0,2.0
6,3.5,5,30.0,5.0
7,2.4,4,25.0,4.0
8,1.2,-2,20.0,2.0
9,1.5,1,5.0,5.0


In [152]:
def regression_tree(Data_set, iterations_vector, x_new):


########################################################################
    
    def s_values(j, Data_set):

        s_values = []

        Xj_sorted = Data_set.sort_values(by=[Data_set.columns[j]], axis=0, ascending=True, ignore_index=True).iloc[:, j]

        for i in range(0, len(Xj_sorted)-1):

            s_values.append( (Xj_sorted[i] + Xj_sorted[i+1] ) / 2  )

        return s_values


########################################################################  

   ## ITERACION 1

    if iterations_vector[0] == 1 : # nacimiento del arbol

        def RSS_1(j,s, Data_set):

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

    ######################################

            y_i_R1 = Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] # y_i tal que x_ij < s

            if len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] ) == 0 :

                y_R1 = 0

            elif len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] ) != 0 : 

                y_R1 = ( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ]  ).sum() / len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ]  )  
   

            ###############

            y_i_R2 = Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] # y_i tal que x_i1 >= 2


            if len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] ) == 0 :

                  y_R2 = 0

            elif len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] ) != 0 : 

                  y_R2 = ( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ]  ).sum() / len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ]  )  

             ##############

            RSS_11 = ( (y_i_R1 - y_R1 )**2 ).sum()

            RSS_12 = ( (y_i_R2 - y_R2 )**2 ).sum()

            RSS_1 = RSS_11 + RSS_12

            ############

            return(RSS_1)

      ################################################

        ## Busqueda de j_star y s_star de la iteracion 1

        RSS_vector = []
        j_vector = []
        s_vector = []

        s_star_vector = []
        j_star_vector = []

        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_1(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})

        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...    
    
    ######################################################################################

    ## ITERACION 2

    if iterations_vector[1] == 2 :  # Desarrollar nodo R1 de la 1ª iteracion

        ################################################################

        def RSS_2(j,s, Data_set):    # Desarrollar nodo R1 de la 1ª iteracion

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

           ################

            y_i_R1 = Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  

            if len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) == 0 :

                y_R1 = 0

            elif len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) != 0 : 

                y_R1 = ( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  )  
   

            ###############

            y_i_R2 = Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  


            if len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) == 0 :

                  y_R2 = 0

            elif len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) != 0 : 

                  y_R2 = ( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  )  

             ##############

            RSS_21 = ( (y_i_R1 - y_R1 )**2 ).sum()

            RSS_22 = ( (y_i_R2 - y_R2 )**2 ).sum()

            RSS_2 = RSS_21 + RSS_22

            ############

            return(RSS_2)


      ###########################################

      # Busqueda de j_star y s_star de la iteracion 2:

        RSS_vector = []
        j_vector = []
        s_vector = []


        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_2(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})



        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...


####################################################################################

  ## ITERACION 3

    if iterations_vector[2] == 3 :  # Desarrollar nodo R2 de la 1ª iteracion

       #########################################

        def RSS_3(j,s, Data_set):    # Desarrollar nodo R2 de la 1ª iteracion

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

           ################

            y_i_R1 = Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  

            if len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ] ) == 0 :

                y_R1 = 0

            elif len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ] ) != 0 : 

                y_R1 = ( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  )  
   

            ###############

            y_i_R2 = Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ]  


            if len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ] ) == 0 :

                  y_R2 = 0

            elif len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ] ) != 0 : 

                  y_R2 = ( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[1]] >= s_star_vector[1]) , 'Y' ]  )  

             ##############

            RSS_31 = ( (y_i_R1 - y_R1 )**2 ).sum()

            RSS_32 = ( (y_i_R2 - y_R2 )**2 ).sum()

            RSS_3 = RSS_31 + RSS_32

            ############

            return(RSS_3)

       ###########################################

      # Busqueda de j_star y s_star de la iteracion 3:

        RSS_vector = []
        j_vector = []
        s_vector = []

 

        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_3(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})

        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...

    ###############################################

    ## Nº de observaciones en cada rama del arbol generado

    obs_rama_1 = len( Data_set.loc[ (Data.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data.iloc[:, j_star_vector[1] ] < s_star_vector[1]) , : ] )
    obs_rama_2 = len( Data_set.loc[ (Data.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data.iloc[:, j_star_vector[1] ] >= s_star_vector[1]) , :])
    obs_rama_3 = len( Data_set.loc[ (Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0]) &  (Data.iloc[:, j_star_vector[2] ] < s_star_vector[2]) , :])
    obs_rama_4 = len( Data_set.loc[ (Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0])  & (Data.iloc[:, j_star_vector[2] ] >= s_star_vector[2]) , :] )

    obs_ramas = [obs_rama_1, obs_rama_2, obs_rama_3, obs_rama_4]


    ## PREDICCION

    if x_new[j_star_vector[0] - 1] < s_star_vector[0] :

        if  x_new[j_star_vector[1] - 1] < s_star_vector[1] :

            y_new_predict = Data_set.loc[ (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) & (Data_set.iloc[:, j_star_vector[1]] < s_star_vector[1]) , 'Y' ]

            y_new_predict = y_new_predict.sum() / len(y_new_predict)

    
    return(j_star_vector, s_star_vector, obs_ramas , y_new_predict) 


Arbol con 3 iteraciones, tiene 4 ramas:  (con k iter tiene k+1 ramas)

rama 1: Data.iloc[:, j_star_vector[0] ] < s_star_vector[0]  & Data.iloc[:, j_star_vector[1] ] < s_star_vector[1]   

rama 2: Data.iloc[:, j_star_vector[0] ] < s_star_vector[0]  & Data.iloc[:, j_star_vector[1] ] >= s_star_vector[1]   

rama 3: Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0]  & Data.iloc[:, j_star_vector[2] ] < s_star_vector[2]   

rama 4: Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0]  & Data.iloc[:, j_star_vector[2] ] >= s_star_vector[2]   

In [134]:
x_new = pd.Series({'X1': -5 , 'X2':3, 'X3':-2})

In [153]:
j_star_vector, s_star_vector, obs_ramas, y_new_predict = regression_tree(Data, range(1,4), x_new)

In [137]:
j_star_vector

[1, 1, 1]

In [138]:
s_star_vector

[6.0, -4.0, -4.0]

In [154]:
obs_ramas

[1, 10, 0, 1]

In [139]:
y_new_predict

2.5